# 7. GUI creation

pyqt and pyqtgraph & pymmcore-widgets

## pyqt and pyqtgraph <a id='pyqt_and_pyqtgraph'></a>

In [4]:
from PyQt5.QtCore import QTimer
from PyQt5.QtWidgets import QApplication, QLabel, QPushButton, QWidget, QGridLayout
from PyQt5.QtGui import QFont
import pyqtgraph as pg
import numpy as np

In [5]:
### simple plot

# start application
app = pg.mkQApp()

# create window
plot_widget = pg.PlotWidget()
x = np.linspace(0, 20, 1000)
y = np.sin(x)
plot_widget.plot(x, y)
plot_widget.show()

# execute application
app.exec()


0

In [6]:
### moving sine curve ('live data acquisition')

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.t = np.linspace(0, 2*np.pi, 1000)
        self.num = 0.01
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph, 0, 0, 1, 1)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.setGeometry(0, 0, 500, 200)
        self.show()

    def update(self):
        self.graph.plotItem.clear()
        data = np.sin(self.t + self.num)
        self.graph.plotItem.plot(self.t, data)
        self.num = self.num + 0.5

app = QApplication(['auto update'])

win = window()

app.exec_()

0

In [ ]:
### display live microscopy images

class window(QWidget):
    def __init__(self):
        super().__init__()
        self.graph = pg.PlotWidget(self)
        self.grid = QGridLayout()
        self.timer = QTimer()
        self.grid.addWidget(self.graph)
        self.setLayout(self.grid)
        self.timer.timeout.connect(self.update)
        self.timer.start(1000) # in ms
        self.graph.show()
        self.show()
    
    def update(self):
        core.setConfig('LED_light', 'on')
        self.graph.plotItem.clear()
        core.snapImage()
        im = core.getImage()
        imv = pg.ImageView()
        imv.setImage(im)
        imv.show()
        #imv.close()
        core.setConfig('LED_light', 'off')

app = QApplication(['auto update'])

win = window()

app.exec_()

In [ ]:
### PyQt5 tutorial

import PyQt5.QtWidgets as qtw
import PyQt5.QtGui as qtg

class MainWindow(qtw.QWidget):
    def __init__(self):
        super().__init__()
        
        # title
        self.setWindowTitle('Microscope Automation')
        
        # layout
        self.setLayout(qtw.QHBoxLayout())
        
        # label
        label = qtw.QLabel('What`s your name?')
        # change font size of label
        label.setFont(qtg.QFont('SansSerif', 30))
        self.layout().addWidget(label)
        
        # create entry box
        entry = qtw.QLineEdit()
        entry.setObjectName('name_field')
        entry.setText('')
        self.layout().addWidget(entry)
        # create a button
        button = qtw.QPushButton('Press me!')
        
        
        self.show()

app = qtw.QApplication([])
mw = MainWindow()

app.exec_()

### pymmcore-widgets

In [ ]:
### some widgets

from qtpy.QtWidgets import QApplication
from pymmcore_widgets import PropertyBrowser, LiveButton, ExposureWidget, ImagePreview
app = QApplication([])

# create a PropertyBrowser widget. By default, this widget will use the active
# Micro-Manager core instance.

pb_widget = PropertyBrowser()
pb_widget.show()

live_btn = LiveButton()
live_btn.show()

exp_wdg = ExposureWidget()
exp_wdg.show()

img_wdg = ImagePreview()
img_wdg.show()

app.exec_()

In [ ]:
### preview widget with buttons

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus

from qtpy.QtWidgets import QApplication, QGroupBox, QHBoxLayout, QVBoxLayout, QWidget

from pymmcore_widgets import (
    ChannelWidget,
    ExposureWidget,
    ImagePreview,
    LiveButton,
    SnapButton,
    ChannelGroupWidget,
)

class ImageFrame(QWidget):
    """An example widget with a snap/live button and an image preview."""

    def __init__(self):
        super().__init__()

        self.preview = ImagePreview()
        self.snap_button = SnapButton()
        self.live_button = LiveButton()
        self.exposure = ExposureWidget()
        self.channel = ChannelWidget()
        self.channel_group = ChannelGroupWidget()

        self.setLayout(QVBoxLayout())

        buttons = QGroupBox()
        buttons.setLayout(QHBoxLayout())
        buttons.layout().addWidget(self.snap_button)
        buttons.layout().addWidget(self.live_button)

        ch_exp = QWidget()
        layout = QHBoxLayout()
        layout.setContentsMargins(0, 0, 0, 0)
        ch_exp.setLayout(layout)

        ch = QGroupBox()
        ch.setTitle("Channel")
        ch.setLayout(QHBoxLayout())
        ch.layout().setContentsMargins(0, 0, 0, 0)
        ch.layout().addWidget(self.channel)
        layout.addWidget(ch)

        ch_gr = QGroupBox()
        ch_gr.setTitle("ChannelGroup")
        ch_gr.setLayout(QHBoxLayout())
        ch_gr.layout().setContentsMargins(0, 0, 0, 0)
        ch_gr.layout().addWidget(self.channel_group)
        layout.addWidget(ch_gr)

        exp = QGroupBox()
        exp.setTitle("Exposure")
        exp.setLayout(QHBoxLayout())
        exp.layout().setContentsMargins(0, 0, 0, 0)
        exp.layout().addWidget(self.exposure)
        layout.addWidget(exp)

        self.layout().addWidget(self.preview)
        self.layout().addWidget(ch_exp)
        self.layout().addWidget(buttons)


if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    #core.loadSystemConfiguration()
    app = QApplication([])
    frame = ImageFrame()
    frame.show()
    #core.snap()
    app.exec_()

![Preview Window](preview_window.png "Title")

##### functions for MainWindow

In [ ]:
### functions

def gauss(x, a, x0, sigma):
        return a * np.exp(-(x - x0) ** 2 / (2 * sigma ** 2))

def _software_autofocus(core, range = 10, step_size = 2):

        #xx, yy = core.getXYPosition()
        z = core.getZPosition()
    
        # define location and type of image saving
        writer = ImageSequenceWriter(r'C:\Users\Admin\Desktop\focus', extension=".png", overwrite=True)
    
        # acquire z-stack
        sequence = MDASequence(
            axis_order="tpgcz",
            stage_positions=[(900, 1214, z)],
            channels=[{'group': 'LED_light', 'config': 'on'}],
            z_plan={'above': range, 'below': range, 'step': step_size}
        )
    
        # run focus mda sequence
        with mda_listeners_connected(writer):
            core.mda.run(sequence)
    
        # calculate focus scores
        focus_images = glob.glob(r'C:\Users\Admin\Desktop\focus\*.png')
        focus_scores = []
        for f in focus_images:
            im = cv2.imread(rf'{f}')
            im_filtered = cv2.medianBlur(im, ksize=3)
            laplacian = cv2.Laplacian(im_filtered, ddepth=cv2.CV_64F, ksize=3)
            focus_score = laplacian.var()
            focus_scores.append(focus_score)
    
        # define x and y for fitting
        y = focus_scores
        x = np.linspace(z-range, z+range, len(y))
    
        # define gauss fit function
        mean = sum(x * y) / sum(y)
        sigma = np.sqrt(sum(y * (x - mean)**2) / sum(y))
    
        # optimize gauss fit
        popt, pcov = curve_fit(gauss, x, y, p0 = [np.max(y), mean, sigma])
    
        # calculate maximum focus score of fit function
        x_fine = np.linspace(z-range, z+range, 100)
        y_max = np.max(gauss(x_fine,*popt))
        pos = np.where(y_max == gauss(x_fine,*popt))
        x_max = x_fine[pos]
        y_max_data = np.max(focus_scores)
        x_max_data = x[np.where(focus_scores == y_max_data)]
    
        # set optimal z position
        core.setPosition(float(x_max[0]))
    
        # remove focus images
        for file in focus_images:
            os.remove(file)

##### MainWindow with multiple widgets

In [2]:
### multiple widgets in main window: live view 

import os
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
from pymmcore_plus import CMMCorePlus
import PyQt5.QtWidgets as qtw
from qtpy.QtWidgets import QStackedWidget, QMainWindow, QAction, QApplication, QGroupBox, QWidget, QHBoxLayout, QVBoxLayout
import pymmcore_widgets as pycw
from useq import MDAEvent, MDASequence, Position
from pymmcore_plus.mda import mda_listeners_connected
from pymmcore_plus.mda.handlers import ImageSequenceWriter
import glob
import cv2
import numpy as np
from scipy.optimize import curve_fit

          
class MainWindow(qtw.QWidget):
    
    def __init__(self):
        
        super().__init__()
        
        self.setLayout(qtw.QHBoxLayout())
        #self.layout = QHBoxLayout()
        #layout.setContentsMargins(0, 0, 0, 0)
        #self.setWindowTitle('Microscope Automation')
        #self.setGeometry(0, 0, 1500, 1000)
        
        snap_button = pycw.SnapButton()
        live_button = pycw.LiveButton()
        image_preview = pycw.ImagePreview()
        channel_group = pycw.ChannelGroupWidget()
        channel = pycw.ChannelWidget()
        
        buttons = QGroupBox()
        buttons.setLayout(qtw.QVBoxLayout())
        #buttons.layout().setContentsMargins(0, 0, 0, 0)
        buttons.layout().addWidget(image_preview)
        buttons.layout().addWidget(snap_button)
        buttons.layout().addWidget(live_button)
        buttons.layout().addWidget(channel_group)
        buttons.layout().addWidget(channel)
        
        autofocus_button = qtw.QPushButton('Autofocus', clicked = lambda: self.software_autofocus(self.core))
        autofocus = QGroupBox() 
        autofocus.setLayout(qtw.QVBoxLayout())
        autofocus.layout().addWidget(autofocus_button)
        
        stage_widget = pycw.StageWidget('XYStage')
        stage = QGroupBox()
        stage.setLayout(qtw.QVBoxLayout())
        stage.layout().addWidget(stage_widget)
        
        self.core = None
        
        self.layout().addWidget(buttons)
        self.layout().addWidget(stage)
        self.layout().addWidget(autofocus)
        
        self.show()
    
    def software_autofocus(self, core, range = 10, step_size = 2):
         _software_autofocus(core, range = range, step_size = step_size)
        

if __name__ == "__main__":
    core = CMMCorePlus()
    core.setDeviceAdapterSearchPaths([mm_dir])
    core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
    app = qtw.QApplication([])
    mw = MainWindow()
    mw.core = core
    app.exec_()

2024-08-29 10:39:51,591 - pymmcore-plus - INFO - (_runner.py:321) MDA Started: stage_positions=(Position(x=900.0, y=1214.0, z=-918.3000000000001, name=None, sequence=None),) channels=(Channel(config='on', group='LED_light', exposure=None, do_stack=True, z_offset=0.0, acquire_every=1, camera=None),) z_plan=ZAboveBelow(go_up=True, above=10.0, below=10.0, step=2.0)
2024-08-29 10:39:51,596 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 0}) channel=Channel(config='on', group='LED_light') x_pos=900.0 y_pos=1214.0 z_pos=-928.3000000000001
2024-08-29 10:39:52,232 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 1}) channel=Channel(config='on', group='LED_light') x_pos=900.0 y_pos=1214.0 z_pos=-926.3000000000001
2024-08-29 10:39:53,006 - pymmcore-plus - INFO - (_runner.py:283) index=mappingproxy({'p': 0, 'c': 0, 'z': 2}) channel=Channel(config='on', group='LED_light') x_pos=900.0 y_pos=1214.0 z_pos=-924.3000000000001
2024-08-

RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 800.

C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
2024-08-29 10:40:17,753 - pymmcore-plus - INFO - (_runner.py:321) MDA Started: stage_positions=(Position(x=900.0, y=1214.0, z=-908.3000000000001, name=None, sequence=None),) channels=(Channel(config='on', group='LED_light', exposure=None, do_stack=True, z_offset=0.0, acquire_every=1, camera=None),) z_plan=ZAboveBelow(go_up=True, above=10.0, below=10.0, 

RuntimeError: Cannot switch camera device while sequence acquisition is running

C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in scalar subtract
  clim_min = (clim_min - range_min) / (range_max - range_min)
C:\Users\Admin\.conda\envs\microscope\lib\site-packages\vispy\visuals\_scalable_textures.py:296: RuntimeWarning: overflow encountered in sca

![Preview Window + stage](preview_window_stage.png)

In [ ]:
### turn LED light off

from pymmcore_plus import CMMCorePlus
mm_dir = 'D:\ProgramFiles\Micro-Manager-2.0'
import os
core = CMMCorePlus()
core.setDeviceAdapterSearchPaths([mm_dir])
core.loadSystemConfiguration(os.path.join(mm_dir, 'MMConfig_Edge42_SOLA_ASIStage_PixelSize.cfg'))
core.setConfig('LED_light', 'off')